In [145]:
import glob, os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

proyectos_df = pd.read_csv('data/proyectos.csv')
evolucion_proyectos_df = pd.read_csv('data/evolucion_proyectos.csv')

In [146]:
evolucion_df = evolucion_proyectos_df.merge(proyectos_df, on='id_proyecto')
evolucion_df.rename(columns={'tipo_x': 'tipo_entrada', 'tipo_y': 'tipo_proyecto'}, inplace=True)

evolucion_df[evolucion_df.tipo_entrada == 'CAMARA_ENTRADA'].groupby('id_proyecto').first().reset_index().groupby(['cuerpo', 'tipo_proyecto']).size().unstack(level=0)


cuerpo,AG,CP,CRR,CSS
tipo_proyecto,,,,
DECLARACION,NaN,NaN,NaN,7.0
LEY,NaN,NaN,59.0,38.0
MINUTA DE COMUNICACION,NaN,NaN,31.0,14.0
PEDIDO DE INFORMES,NaN,3.0,1784.0,408.0
PROYECTO DE DECLARACION,NaN,NaN,3.0,4.0
PROYECTO DE LEY,NaN,NaN,385.0,155.0
PROYECTO DE MINUTA DE COMUNICACION,NaN,NaN,27.0,8.0
PROYECTO DE RESOLUCION,12.0,3.0,21.0,14.0
RESOLUCION,4.0,NaN,36.0,30.0


In [147]:
evolucion_df[evolucion_df.tipo_entrada.isin(['CAMARA_APRUEBA', 'CAMARA_RECHAZA'])].groupby('id_proyecto').first().reset_index().groupby(['cuerpo', 'tipo_proyecto']).size().unstack(level=0)


cuerpo,AG,CRR,CSS
tipo_proyecto,,,
DECLARACION,NaN,NaN,7.0
LEY,NaN,59.0,38.0
MINUTA DE COMUNICACION,NaN,31.0,14.0
PROYECTO DE LEY,NaN,27.0,11.0
PROYECTO DE MINUTA DE COMUNICACION,NaN,1.0,NaN
PROYECTO DE RESOLUCION,NaN,2.0,3.0
RESOLUCION,4.0,37.0,30.0


In [148]:
evolucion_df[(evolucion_df.tipo_entrada == 'PROMULGA_PE')].groupby('id_proyecto').first().reset_index().groupby(['cuerpo', 'tipo_proyecto']).size()


cuerpo  tipo_proyecto
CRR     LEY              24
CSS     LEY              73
dtype: int64

In [153]:
tiempo_evolucion = evolucion_df[evolucion_df.tipo_entrada == 'CAMARA_ENTRADA'][['id_proyecto', 'tipo_proyecto', 'fecha', 'tipo_entrada']].merge(evolucion_df[evolucion_df.tipo_entrada.isin(['CAMARA_APRUEBA', 'CAMARA_RECHAZA'])][['id_proyecto', 'fecha', 'tipo_entrada']], on='id_proyecto')
tiempo_evolucion.loc[:, 'duracion'] = tiempo_evolucion.apply(lambda x : datetime.datetime.strptime(x['fecha_y'], '%Y-%m-%d') - datetime.datetime.strptime(x['fecha_x'], '%Y-%m-%d'), axis=1)
tiempo_evolucion


,id_proyecto,tipo_proyecto,fecha_x,tipo_entrada_x,fecha_y,tipo_entrada_y,duracion
0,132477,DECLARACION,2016-11-01,CAMARA_ENTRADA,2016-11-01,CAMARA_APRUEBA,0 days
1,132477,DECLARACION,2016-11-01,CAMARA_ENTRADA,2016-11-01,CAMARA_APRUEBA,0 days
2,133188,RESOLUCION,2016-12-14,CAMARA_ENTRADA,2016-12-14,CAMARA_APRUEBA,0 days
3,133188,RESOLUCION,2016-12-14,CAMARA_ENTRADA,2016-12-14,CAMARA_APRUEBA,0 days
4,132117,LEY,2016-10-18,CAMARA_ENTRADA,2016-11-23,CAMARA_APRUEBA,36 days
5,132117,LEY,2016-10-18,CAMARA_ENTRADA,2016-12-14,CAMARA_APRUEBA,57 days
6,132117,LEY,2016-12-06,CAMARA_ENTRADA,2016-11-23,CAMARA_APRUEBA,-13 days
7,132117,LEY,2016-12-06,CAMARA_ENTRADA,2016-12-14,CAMARA_APRUEBA,8 days
8,132117,LEY,2016-10-18,CAMARA_ENTRADA,2016-11-23,CAMARA_APRUEBA,36 days
9,132117,LEY,2016-10-18,CAMARA_ENTRADA,2016-12-14,CAMARA_APRUEBA,57 days
